In [7]:
%%time
print('importing modules...')
import tensorflow as tf
import keras
import sys, os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from imageio import imread
from PIL import Image
from random import randint
sys.path.append(os.pardir)
from ssd_vgg16 import SSD

annotations = {'Car':100,'Truck':100,'Pedestrian':100,'stop_sign':100}
fmap = {'conv4_3_norm': [1.0, 2.0, 1.0/2.0],
        'fc7': [1.0, 2.0, 3.0, 1.0/2.0, 1.0/3.0],
        'conv6_2': [1.0, 2.0, 3.0, 1.0/2.0, 1.0/3.0],
        'conv7_2': [1.0, 2.0, 3.0, 1.0/2.0, 1.0/3.0],
        'conv8_2': [1.0, 2.0, 1.0/2.0],
        'conv9_2': [1.0, 2.0, 1.0/2.0]}

input_shape = (300, 300, 3)
NUM_CLASSES = len(annotations) + 1

print("generating model")
model = SSD((300,300,3), num_classes=NUM_CLASSES, fmap=fmap)
print("model created")

importing modules...
generating model
model created
CPU times: user 697 ms, sys: 21.2 ms, total: 718 ms
Wall time: 737 ms


In [4]:
# Make data and prepare for training
img_folderpath = 'data/img/'
csv_filepath = 'data/alldata.csv'

print('making data...')
csv = pd.read_csv(csv_filepath, index_col=0)

print('picking files randomly...')
use_imgfile_names = pd.Series()
for i in range(len(annotation)):
    tmp = csv.filename[csv.object.isin([[x for x in annotation.keys()][i]])].sample(n=[v for v in annotation.values()][i])
    use_imgfile_names = use_imgfile_names.append(tmp)

print('reading image files...')
annotation_list = []
imagesize_list = []
for name in use_imgfile_names:
    try:
        im = Image.open(img_folderpath + name)
        t = list(im.size)
        t.insert(0,name)
        imagesize_list.append(t)
    except FileNotFoundError:
        pass

print('concatenating...')
csv = csv[csv.filename.isin(use_imgfile_names) & csv.object.isin(annotation.keys())]
imgsize = pd.DataFrame(imagesize_list).rename(index=str, columns={0:'filename', 1:'w', 2:'h'})
df = pd.merge(csv, imgsize)
df.xmin /= df.w
df.ymin /= df.h
df.xmax /= df.w
df.ymax /= df.h
df['bounding_box'] = df[['xmin','ymin','xmax','ymax']].values.tolist()
one_hot_category = []
for i,row in df.iterrows():
    one_hot = [0] * len(annotation)
    pos = [k for k,val in enumerate(annotation) if val == row.object][0]
    one_hot[pos] = 1
    one_hot_category.append(one_hot)
df['one_hot_category'] = one_hot_category
df['one_hot_vector'] = df.bounding_box + df.one_hot_category
df = pd.DataFrame(df.groupby(['filename','w','h']).one_hot_vector.apply(list)).reset_index()
gt = dict()
for i,row in df.iterrows():
    gt[row.filename] = np.array(row.one_hot_vector)

print('generating train / validation files...')
keys = sorted(gt.keys())
num_train = int(round(0.8 * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)

inputs = []
img_path = img_folderpath + sorted(val_keys)[0]
img = image.load_img(img_path, target_size=(input_shape[1],input_shape[0]))
img = image.img_to_array(img)
inputs.append(img.copy())
inputs = preprocess_input(np.array(inputs))
print('done.')

In [ ]:
# subsample weights (optional)
weights_filepath = 'VGG_coco_SSD_300x300_iter_400000_subsampled_4_classes.h5'

In [ ]:
# training config



In [ ]:
# train

In [ ]:
scale=[0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05], # MS_COCO